In [1]:
#!/usr/bin/env python
#SBATCH --job-name=PSNN
#SBATCH --error=%x.%j.err
#SBATCH --output=%x.%j.out
#SBATCH --mail-user=hzhao@teco.edu
#SBATCH --export=ALL
#SBATCH --time=48:00:00
#SBATCH --partition=sdil
#SBATCH --gres=gpu:1

import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'utils'))
from configuration import *
import torch
import pprint
from utils import *
import BaselineModels as B

In [2]:
args = parser.parse_args([])
args.task = 'temporized'
args.SEED = 1
args.metric = 'acc'
args.lnc = True

In [3]:
args = FormulateArgs(args)
print(f'Training network on device: {args.DEVICE}.')
MakeFolder(args)

Training network on device: cuda.


In [4]:
train_loader, datainfo = GetDataLoader(args, 'train')
valid_loader, datainfo = GetDataLoader(args, 'valid')
test_loader, datainfo = GetDataLoader(args, 'test')
pprint.pprint(datainfo)

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 32,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}


In [5]:
SetSeed(args.SEED)
setup = f"baseline_model_SNN_data_{datainfo['dataname']}_seed_{args.SEED}.model2"
print(f'Training setup: {setup}.')

Training setup: baseline_model_SNN_data_acuteinflammation_seed_1.model2.


In [6]:
msglogger = GetMessageLogger(args, setup)
msglogger.info(f'Training network on device: {args.DEVICE}.')
msglogger.info(f'Training setup: {setup}.')
msglogger.info(datainfo)

In [7]:
topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
msglogger.info(f'Topology of the network: {topology}.')

mlp = B.mlp(args, topology).to(args.DEVICE)
lstm = B.lstm(args, datainfo['N_feature'], datainfo['N_class']).to(args.DEVICE)
pnn = B.pNN(topology, args).to(args.DEVICE)
snn = B.SpikingNeuralNetwork(args, topology).to(args.DEVICE)

In [8]:
for n, p in snn.named_parameters():
    print(n)
len(snn.GetParam())

beta
threshold
model.MAC0.weight
model.MAC1.weight


4

In [19]:
for n, p in mlp.named_parameters():
    print(n)
len(mlp.GetParam())

activation.eta
model.0-th pLayer.weight
model.0-th pLayer.bias
model.1-th pLayer.weight
model.1-th pLayer.bias


4

In [20]:
for n, p in lstm.named_parameters():
    print(n)
len(lstm.GetParam())

rnn.activation.eta
rnn.sigmoid.eta
rnn.lstm_cell.i2h.weight
rnn.lstm_cell.i2h.bias
rnn.lstm_cell.h2h.weight
rnn.lstm_cell.h2h.bias


4

In [21]:
for n, p in pnn.named_parameters():
    print(n)
len(pnn.GetParam())

act.rt_
act.eta_estimator.0-MAC.weight
act.eta_estimator.0-MAC.bias
act.eta_estimator.0-ACT.weight
act.eta_estimator.1-MAC.weight
act.eta_estimator.1-MAC.bias
act.eta_estimator.1-ACT.weight
act.eta_estimator.2-MAC.weight
act.eta_estimator.2-MAC.bias
act.eta_estimator.2-ACT.weight
act.eta_estimator.3-MAC.weight
act.eta_estimator.3-MAC.bias
act.eta_estimator.3-ACT.weight
act.eta_estimator.4-MAC.weight
act.eta_estimator.4-MAC.bias
act.eta_estimator.4-ACT.weight
act.eta_estimator.5-MAC.weight
act.eta_estimator.5-MAC.bias
act.eta_estimator.5-ACT.weight
act.eta_estimator.6-MAC.weight
act.eta_estimator.6-MAC.bias
act.eta_estimator.6-ACT.weight
act.eta_estimator.7-MAC.weight
act.eta_estimator.7-MAC.bias
act.eta_estimator.7-ACT.weight
act.eta_estimator.8-MAC.weight
act.eta_estimator.8-MAC.bias
act.eta_estimator.8-ACT.weight
act.eta_estimator.9-MAC.weight
act.eta_estimator.9-MAC.bias
act.eta_estimator.9-ACT.weight
act.eta_estimator.10-MAC.weight
act.eta_estimator.10-MAC.bias
act.eta_estimator.10

2

In [19]:
a, b = snn(x)
b.sum().backward()

In [20]:
for n, p in snn.getAllNamedParameters():
    print(n, p.grad)

weight tensor([[ 4.7734e-01,  8.0344e+00, -1.6130e+03],
        [ 8.4351e-02,  4.4608e-01, -4.1365e+02],
        [ 5.8983e-01,  1.2794e+01, -1.7606e+03],
        [ 7.4504e-01,  4.5500e+00, -3.5763e+03],
        [ 4.3930e-01,  1.3040e+00, -1.7630e+03],
        [ 3.9155e-01,  4.0067e+00, -2.7251e+03],
        [ 1.2602e+00,  1.7402e+01, -3.6078e+03]], device='cuda:0')
weight tensor([[    0.0000,     0.0000],
        [    0.0000,     0.0000],
        [  684.3574,   684.3574],
        [22224.3848, 22224.3848]], device='cuda:0')
beta tensor(-896.5642, device='cuda:0')
threshold tensor(345.7404, device='cuda:0')


In [21]:
snn.GetParam()

[Parameter containing:
 tensor([[-0.0690,  0.0418, -0.0213],
         [-0.0840, -0.0420, -0.0964],
         [ 0.0142, -0.0164, -0.0358],
         [-0.0059, -0.2492,  0.0242],
         [ 0.0288,  0.0103,  0.1100],
         [-0.0342,  0.0947,  0.0622],
         [-0.0448, -0.0286,  0.0388]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[ 0.1595, -0.1528],
         [ 0.1016, -0.0202],
         [-0.1286,  0.0823],
         [-0.0610, -0.1296]], device='cuda:0', requires_grad=True),
 tensor(2.9444, device='cuda:0', grad_fn=<CopyBackwards>),
 tensor(1., device='cuda:0', grad_fn=<CopyBackwards>)]

In [22]:
optimizer = torch.optim.Adam(snn.GetParam(), lr=args.LR)

ValueError: can't optimize a non-leaf Tensor

In [12]:
for x, y in train_loader:
    pass

In [13]:
x.shape

torch.Size([70, 6, 32])

In [14]:
yyy = lstm(x)
yyy.shape

torch.Size([70, 2])

In [15]:
if os.path.isfile(f'{args.savepath}/{setup}'):
    print(f'{setup} exists, skip this training.')
    msglogger.info('Training was already finished.')
else:
    topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
    msglogger.info(f'Topology of the network: {topology}.')

    
    lossfunction = B.CELOSS().to(args.DEVICE)
    optimizer = torch.optim.Adam(snn.parameters(), lr=args.LR)

    if args.PROGRESSIVE:
        lstm, best = train_pnn_progressive(lstm, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
    else:
        lstm, best = train_pnn(lstm, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

    if best:
        if not os.path.exists(f'{args.savepath}/'):
            os.makedirs(f'{args.savepath}/')
        torch.save(lstm, f'{args.savepath}/{setup}')
        msglogger.info('Training if finished.')
    else:
        msglogger.warning('Time out, further training is necessary.')

| Epoch:      0 | Train loss: 6.7707e-01 | Valid loss: 6.8251e-01 | Train acc: 0.4857 | Valid acc: 0.3913 | patience:   0 | lr: 0.1 | Epoch time: 0.1 | Power: 0.00e+00 |
| Epoch:      1 | Train loss: 6.7707e-01 | Valid loss: 6.8251e-01 | Train acc: 0.4857 | Valid acc: 0.3913 | patience:   1 | lr: 0.1 | Epoch time: 0.1 | Power: 0.00e+00 |
| Epoch:      2 | Train loss: 6.7707e-01 | Valid loss: 6.8251e-01 | Train acc: 0.4857 | Valid acc: 0.3913 | patience:   2 | lr: 0.1 | Epoch time: 0.1 | Power: 0.00e+00 |
| Epoch:      3 | Train loss: 6.7707e-01 | Valid loss: 6.8251e-01 | Train acc: 0.4857 | Valid acc: 0.3913 | patience:   3 | lr: 0.1 | Epoch time: 0.1 | Power: 0.00e+00 |
| Epoch:      4 | Train loss: 6.7707e-01 | Valid loss: 6.8251e-01 | Train acc: 0.4857 | Valid acc: 0.3913 | patience:   4 | lr: 0.1 | Epoch time: 0.1 | Power: 0.00e+00 |
| Epoch:      5 | Train loss: 6.7707e-01 | Valid loss: 6.8251e-01 | Train acc: 0.4857 | Valid acc: 0.3913 | patience:   5 | lr: 0.1 | Epoch time: 0.1 


KeyboardInterrupt

